# Perceptron

In this session we will implement the Perceprton algorithm from scratch. This notebook contains the following parts:

1) Reading the data

2) Vector operations

3) Implementing the Perceptron algorithm

## 1) Reading the data

The data we will use to test our implementation is a collection of movie reviews, each associated with a rating. The data comes in a pre-processed form, with the features extracted, and has the following string format:

```
target feature_1:feature_value1 features_2:value_2 ...

```

For example:

```
1 0:2.0 3:4.0 123:1.0
```
This means the example's target label is 1, features 0 is 2.0, feature
3 is 4.0, feature 123 is 1.0 (all the other features are implicitly
0.0). The features are word counts.

Let us first print the first few lines of the data file in order to check the format. The cell below reads and prints the first 3 lines of the data.

In [73]:
for index, line in enumerate(open('sentiment.feat')):
    if index < 3:
        print(line)

9 0:9 1:1 2:4 3:4 4:6 5:4 6:2 7:2 8:4 10:4 12:2 26:1 27:1 28:1 29:2 32:1 41:1 45:1 47:1 50:1 54:2 57:1 59:1 63:2 64:1 66:1 68:2 70:1 72:1 78:1 100:1 106:1 116:1 122:1 125:1 136:1 140:1 142:1 150:1 167:1 183:1 201:1 207:1 208:1 213:1 217:1 230:1 255:1 321:5 343:1 357:1 370:1 390:2 468:1 514:1 571:1 619:1 671:1 766:1 877:1 1057:1 1179:1 1192:1 1402:2 1416:1 1477:2 1940:1 1941:1 2096:1 2243:1 2285:1 2379:1 2934:1 2938:1 3520:1 3647:1 4938:1 5138:4 5715:1 5726:1 5731:1 5812:1 8319:1 8567:1 10480:1 14239:1 20604:1 22409:4 24551:1 47304:1

7 0:7 1:4 2:2 3:2 5:4 6:1 8:2 9:2 14:1 16:1 18:1 20:1 22:1 24:1 27:1 29:1 34:1 36:3 37:2 41:1 42:1 48:3 52:2 79:2 91:1 100:1 106:1 119:1 154:1 166:1 172:1 207:1 288:1 321:1 336:1 353:1 365:1 383:1 386:2 390:1 429:1 504:1 517:1 706:1 729:1 748:1 911:1 1107:1 1195:1 1332:1 1433:1 1620:1 1662:1 2047:1 2365:1 2390:1 2673:1 3111:1 3230:1 3513:1 4065:1 4357:1 5138:1 5365:1 5884:1 6617:1 11023:1 11392:1 11798:1 12318:1 12831:1 13645:1 18316:1 22409:1 22496:1 2724

We will start by writing some code to work with this data.


## Exercise 1 _ Binarize

We want to work with binary labels (positive vs negative) but we have integer ratings (0 to 10). We will convert the ratings to binary labels using 5 as a threshold: is the rating is higher, the label will be positive, otherwise it'll be negative.

Define function ``binarize``. The function should accept a list of numeric
ratings, and return a list of class labels -1 and 1.


In [74]:
def binarize(y):
    return [1 if i>5 else -1 for i in y]

y = list(range(0,11))
print(binarize(y))

[-1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1]


## Exercise 2 _ Parse line

Define function ``parse_line`` which extracts the target label and
feature values of a training example from a string. Your function should return two values:

- a dictionary mapping features (as ints) to values (as floats)
- the target label (as int)

In [75]:
def parse_line(line):
    def parse_feature(x):
        key, val = x.split(':')
        return(int(key), float(val))
    fields=line.split()
    y = int(fields[0])
    x = dict([parse_feature(i) for i in fields[1:]])
    return x,y
    
line = '1 0:2.0 3:4.0 123:1.0\n'
print(parse_line(line))

({0: 2.0, 3: 4.0, 123: 1.0}, 1)


Once we have these two functions, we can read our dataset from the input file.

In [76]:
import random
random.seed(123)
def prepare_data():
    """Read data from input file, shuffle and return a list of training examples."""
    # Sample 10% of data so we don't crash the server
    train = []
    for line in open('sentiment.feat'):
        if random.random() >= 0.9:
            train.append(parse_line(line))
    
    # We will shuffle the examples so that we have a mixture of positive and negative cases
    random.shuffle(train)
    # We unzip the data into input features and target labels
    X, Z = list(zip(*train))
    # We binarize the labels
    Y = binarize(Z)
    return list(zip(X,Y))

In [77]:
XY = prepare_data()
print("We have {} examples".format(len(XY)))

We have 2547 examples


The variable ``XY`` contains the list of examples, where each example is a 
tuple with the input in the first position and the target in the second position.


## 2) Vector operations

Our inputs are word counts and therefore sparse: most feature values are zero. For this reason we are representing these feature vectors as Python dictionaries where we will record the non-zero values, and treat the zero values as implicit. We now need to implement the vector operations needed for the Perceptron algorithm for the dictionary representation.

## Exercise 3 _ Dot product

Define the function ``dot`` which calculates the dot (or inner) product of two
vectors. This function should work on vectors represented as
dictionaries: any missing key in the dictionary is implicitly equal to
0.0. In order to compute the dot product, you need to multiply the
values at the corresponding keys together, and sum all the results.
This function can assume that the vector with more non-zero entries
(i.e. dictionary with more keys) will be the first argument. This
is useful for efficiency.


In [78]:
def dot(big, small):
    s= 0.0
    for k,v in small.items():
        s = s+v * big.get(k,0)
    return s

u = {0:0.5, 1:2.0, 2:-2.5}
v = {0:-0.5, 2:2.5, 3:0.5}
print(dot(u, v))

-6.5


## Exercise 4 _ Increment

Define function ``increment`` which modifies a vector by adding
another vector to it. The two vectors are given as dictionaries:

- `u` - the vector to be modified (as a dictionary)
- `v` - the vector (as a dictionary) which should be added to `u` 

This function should not return anything, but it should modify `u` so
that it contains the union of the keys present in the two vectors. The
value at each key should be the sum of the values at this key in the
two vectors. Remember that if a key is missing from the dictionary
representing the vector, the value is implicitly equal to 0.0.

Note: a function (like ``increment``) which changes one of its inputs is called **destructive**.

In [79]:
def increment(big, small):
    for k,v in small.items():
        big[k] = big.get(k,0.0) + v
        
        
u = {0:0.5, 1:2.0, 2:-2.5}
v = {0:-0.5, 2:2.5, 3:0.5}
increment(u, v)
print(u) 
u = {0:0.5, 1:2.0, 2:-2.5}
w = {}
increment(u, w)
print(u)


{0: 0.0, 1: 2.0, 2: 0.0, 3: 0.5}
{0: 0.5, 1: 2.0, 2: -2.5}


## Exercise 5 _ Scale

Define function ``scale`` which takes a vector `u` (as a dictionary)
and a number `n`, and returns a new vector dictionary which contains
the values in vector `u` multiplied by `n`. This function should not
modify its arguments, but should return a new dictionary. Note: the function
``increment`` combined with the function ``scale`` can be used to
represent vector substraction (``decrement``).

In [80]:
def scale(u, n):
    results = {}
    for k, v in u.items():
        results[k] = v*n
    return results

u = {0:0.5, 1:2.0, 2:-2.5}
v = {0:-0.5, 2:2.5, 3:0.5}
n = 2.0
print(scale(u, n))
print(u) # u should be unchanged
u = {0:0.5, 1:2.0, 2:-2.5}
v = {0:-0.5, 2:2.5, 3:0.5}
increment(u, scale(v, -1.0))
print(u)


{0: 1.0, 1: 4.0, 2: -5.0}
{0: 0.5, 1: 2.0, 2: -2.5}
{0: 1.0, 1: 2.0, 2: -5.0, 3: -0.5}


## 3) Implementing the Perceptron algorithm

We will now start implementing the Perceptron algorithm. We will use a dictionary to keep the weights and the bias of the model. We'll initialize the bias to zero, and the weights to a vector of all zeros (represented as an empty dictionary). During training, we will update the values as we learn.


In [81]:
def initialize():
    w = {}
    b = 0.0
    return {'w':w,'b':b}

model = initialize()

## Exercise 6 _ Predict

Define function ``predict`` which takes two arguments: 

- `model` - the dictionary representation of the perceptron model with
  keys 'w' for weights and 'b' for the bias
- `x` - new input (as a dictionary)

It should return the predicted target for the input `x`: it should
compute the discriminant function `wx + b` and predict 1 if it is
greater than or equal to 0, and -1 otherwise.



In [82]:
def predict(model, x):
    gx = dot(model['w'],x) + model['b']
    return 1 if gx>=0 else -1
    

x = u = {0:0.5, 1:2.0, 2:-2.5}
model = initialize()
print(predict(model, x))

1


## Exercise 7 _ Update

We will now implement the update functionality of the perceptron
algorithm. You need to code the function ``update``, which is given a
training example, and first uses the ``predict`` function to guess
the target, then updates the weights and the bias of the model
depending on whether the guess is correct or incorrect, and on the
direction of the mistake. Finally, the function should return the
guess. ``update`` is given two arguments:

- `model` - this is the dictionary with keys 'w' (with weights) and
  'b' (with the bias)
- `xy` - this is the pair `(x,y)` where `x` is the input vector (as a
  dictionary) and `y` is the target (-1 or 1, as an int).

Details of the perceptron update rule are shown in the lecture slides
for Session 4.

Hints:

- You can use the function  ``predict`` to make the guess.
- When updating the weights, use the function ``increment``
    (possibly with combination with ``scale``) to add the example
    input to (or subtract it from) the model weights.

Note: this function is destructive because it modifies its first argument.

In [83]:
def update(model, xy):
    x,y=xy
    y_pred = predict(model, x)
    if y_pred==1 and y==-1:
        increment(model['w'], scale(x,-1))
        model['b'] = model['b']-1
    elif y_pred==-1 and y==1:
        increment(model['w'], x)
        model['b'] = model['b'] +1
    return y_pred

x = { 0: 7.0, 1: 4.0, 3: 4.0, 4: 2.0, 5: 2.0, 11: 3.0 }
y = -1
model = initialize()
y_pred = update(model, (x,y))
print(y_pred)
print(model)

1
{'w': {0: -7.0, 1: -4.0, 3: -4.0, 4: -2.0, 5: -2.0, 11: -3.0}, 'b': -1.0}


## Exercise 8 _ Learn

Now you'll implement the function ``learn`` will processes each
training example, generates a guess, and makes an update (using the
``update`` function from Exercise 7). Finally it will return the list of
guesses made. This function is given 2 arguments:

- `model` - the dictionary representing the perceptron model
- `XY` - the list of the training examples, where each example is a
  tuple `(x, y)`, `x` being the input vector dictionary and `y` the
  target (1 or -1)


You can implement this function following these steps:

- Initialize the list of guesses to an empty list
- For each training example `(x,y)`
  
   - get a guess using the ``update`` function with the `model`
   - add this guess to the list of guesses
   
- Return the complete list of guesses.


In [84]:
def learn(model, XY):
    predictions = []
    for i in range(len(XY)):
        x_i,y_i =XY[i]
        y_pred =update(model, (x_i,y_i))
        predictions.append(y_pred)
    return predictions

## Exercise 9 _ Evaluate

In order to test our algorithm we will define function ``evaluate``, which takes the list of true class labels,
another list with predicted class labels, and returns a tuple with
three elements:

- total number of errors
- total number of labels  
- error rate

In [85]:
def evaluate(gold, predicted):
    N = len(gold)
    errors = sum(1 if p!=y else 0 for p,y in zip(predicted,gold))
    return(errors, N, errors / N)
y_true = [-1,-1,1,1,1]
y_pred = [-1,1,1,1,-1]
print(evaluate(y_true, y_pred))
print(evaluate(y_true, y_true))


(2, 5, 0.4)
(0, 5, 0.0)


## Running

Let's do a pass of perceptron learnng over the first 2000 examples.

In [86]:
XY_train = XY[:2000]
Y_true = [ xy[1] for xy in XY_train ]
model = initialize()
Y_pred = learn(model, XY_train)

Let's check how good the guesses made during training were.

In [88]:
print(evaluate(Y_true, Y_pred))

(731, 2000, 0.3655)


## Multiple passes

We can run the model over the data a few times and monitor the performance on the second part of the data.


In [89]:
def run(XY, passes=1):
    XY_train = XY[:2000]
    XY_dev   = XY[2000:]
    Y_train = [ xy[1] for xy in XY_train ]
    Y_dev   = [ xy[1] for xy in XY_dev ]
    model = initialize()
    print("{:>3s} {:>7s} {:>7s}".format("Pass", "err_tr", "err_dev"))
    for i in range(1, passes+1):
        predicted_train = learn(model, XY_train)
        _, _, rate_train = evaluate(Y_train, predicted_train)
        predicted_dev = [ predict(model, x) for (x,_) in XY_dev ]
        _, _, rate_dev = evaluate(Y_dev, predicted_dev)
        print("{:3d} {:7.3f} {:7.3f}".format(i, rate_train, rate_dev))
        
run(XY, passes=5)        

Pass  err_tr err_dev
  1   0.365   0.298
  2   0.224   0.272
  3   0.187   0.254
  4   0.154   0.194
  5   0.139   0.199
